In [ ]:
# Make this change to allow code DensePhrases to run from a jupyter notebook
# L227 in DPhrases/options.py:
#     opt, unknown = self.parser.parse_known_args()  # opt = self.parser.parse_args()

In [1]:
# Set environment variables

%env BASE_DIR=../
%env DATA_DIR=../densephrases-data
%env SAVE_DIR=../outputs
%env CACHE_DIR=../cache

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from integrate import *

env: BASE_DIR=../
env: DATA_DIR=../densephrases-data
env: SAVE_DIR=../outputs
env: CACHE_DIR=../cache


`fused_weight_gradient_mlp_cuda` module not found. gradient accumulation fusion with weight gradient computation disabled.


In [70]:
# EDIT THIS: Set inference parameters

params = {
    "top_k": 5,
    "use_large_index": True,
    "strip_qmark": False,
    "strip_qword1": True,
    "strip_qword2": False,
    "strip_qword_mode": "all",  # first / all
    "prepend_hop_phrase": False,
    "retrieval_unit": "phrase",  # phrase / sentence / paragraph
    "single_hop": False
}

batch_size = 100

In [6]:
# Load the DensePhrases module
print("Loading DensePhrases module...")
model = load_densephrase_module(load_dir=load_dir, 
                                dump_dir=dump_dir, 
                                index_name=idx_name.replace('_small', ('' if params["use_large_index"] \
                                                                       else '_small')), 
                                device=device)

Loading DensePhrases module...
This could take up to 15 mins depending on the file reading speed of HDD/SSD


Downloading:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

Loading DensePhrases Completed!


In [71]:
# Read questions and answers from data_path
queries = read_queries(data_path)
questions = get_key(queries, 'question')
answers = get_key(queries, 'answer')
answers = [answer[0] for answer in answers]  # flattening

# Setup function arguments based on the parameters
method = 'pre' if params["prepend_hop_phrase"] else 'post'
top_k = params["top_k"]
ret_unit1 = params["retrieval_unit"]
strip_ques1 = params["strip_qmark"]
strip_prompt1 = params["strip_qword1"]
strip_ques2 = params["strip_qmark"]
strip_prompt2 = params["strip_qword2"]
strip_prompt_mode = params["strip_qword_mode"]
single_hop = params["single_hop"]

Loading data from /gypsum/scratch1/dagarwal/multihop_dense_retrieval/data/hotpot/hotpot_qas_val.json
Found 5918 bridge questions in 7405 total questions.


In [72]:
DEBUG=False
if DEBUG:
    ques = questions[:20]
else:
    ques = questions
    
# Run batched inference
results = []
print("Running batched multi-hop inference...")
for i in tqdm(range(0, len(ques), batch_size)):
    batch_results = run_batch_inference(model, ques[i:i + batch_size], strip_ques1, strip_prompt1,
                                        strip_ques2, strip_prompt2, ret_unit1, ret_unit2, ques_terms, method,
                                        strip_prompt_mode, answers=answers[i:i + batch_size], write=False, 
                                        top_k=top_k, silent=True, single_hop=single_hop)
    results += batch_results

Running batched multi-hop inference...


  0%|                                                                                                   | 0/60 [00:51<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1.14 GiB (GPU 0; 10.76 GiB total capacity; 3.72 GiB already allocated; 225.44 MiB free; 4.99 GiB reserved in total by PyTorch)

In [ ]:
# Write predictions to disk
run_id = __import__("calendar").timegm(__import__("time").gmtime())
if not single_hop:
    out_file = f'predictions_{run_id}.json'
else:
    out_file = f'singlehop_{run_id}.json'
meta_out_file = out_file.replace('.json', '_meta.json')
with open(out_file, 'w') as fp:
    json.dump(results, fp, indent=4)
with open(meta_out_file, 'w') as fp:
    json.dump(params, fp, indent=4)
print(f"Predictions saved at {out_file}")
print(f"Run metadata saved at {meta_out_file}")

In [ ]:
!ls